# generic space

In [ ]:
import nni.retiarii.nn.pytorch as nn
import torch
from collections import OrderedDict

from nni import trace
from nni.retiarii import model_wrapper


In [ ]:
@model_wrapper
class SimpleAutoencoderSpace(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),
        )

        out_layers = OrderedDict(
            [
            ("RelU", nn.Sequential(
                nn.ConvTranspose2d(1, 1, 3, stride=2, padding=1, output_padding=1),
                nn.ReLU())),
            
            ("Sigmoid", nn.Sequential(
                nn.ConvTranspose2d(1, 1, 3, stride=2, padding=1, output_padding=1),
                nn.Sigmoid())),
            ("SiLU",  nn.Sequential(
                nn.ConvTranspose2d(1, 1, 3, stride=2, padding=1, output_padding=1),
                nn.SiLU())),
            ]
        )

        self.out = nn.LayerChoice(out_layers,label='out')


    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        x = self.out(x)
        return x

# multi-trial search

In [ ]:
from nni.experiment import Experiment
from nni.retiarii.evaluator import FunctionalEvaluator
from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig
from nni.retiarii.strategy import DARTS, Random

from darts.eval_lightning import LightningEval
from darts.phantom import generate_phantom

In [ ]:
# input image (phantom)
resolution = 7
phantom = generate_phantom(resolution=resolution)

# search space
model_space = SimpleAutoencoderSpace()

evaluator = LightningEval(
                model_space,
                buffer_size=100,
                num_iter=50,
                lr=.0005,
                noise_type='gaussian',
                noise_factor=0.15,
                resolution=resolution,
                )

# search strategy
search_strategy = Random(dedup=True)

# experiment
exp = RetiariiExperiment(model_space, evaluator, [], search_strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = 'mnist_search'
exp_config.trial_code_directory = 'C:/Users/Public/Public_VS_Code/NAS_test'
exp_config.experiment_working_directory = 'C:/Users/Public/nni-experiments'

exp_config.max_trial_number = 12   # spawn 50 trials at most
exp_config.trial_concurrency = 2  # will run two trials concurrently

exp_config.trial_gpu_number = 1 # will run 1 trial(s) concurrently
exp_config.training_service.use_active_gpu = True

# Execute
exp.run(exp_config, 8081)

In [ ]:
experiment = Experiment.connect(8081)
experiment.stop()

# one-shot

In [ ]:
import torch

from nni import trace
from nni.retiarii import model_wrapper

import nni.retiarii.nn.pytorch as nn

from collections import OrderedDict
from nni.experiment import Experiment
from nni.retiarii.evaluator import FunctionalEvaluator
from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig
from nni.retiarii.strategy import DARTS

from darts.eval_lightning import LightningEval
from darts.phantom import generate_phantom


In [ ]:
@model_wrapper
class SimpleAutoencoderSpace(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),
        )

        out_layers = OrderedDict(
            [
            ("RelU", nn.Sequential(
                nn.ConvTranspose2d(1, 1, 3, stride=2, padding=1, output_padding=1),
                nn.ReLU())),
            
            ("Sigmoid", nn.Sequential(
                nn.ConvTranspose2d(1, 1, 3, stride=2, padding=1, output_padding=1),
                nn.Sigmoid())),
            ("SiLU",  nn.Sequential(
                nn.ConvTranspose2d(1, 1, 3, stride=2, padding=1, output_padding=1),
                nn.SiLU())),
            ]
        )

        self.out = nn.LayerChoice(out_layers,label='out')


    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        x = self.out(x)
        return x

In [ ]:
# input image (phantom)
resolution = 7
phantom = generate_phantom(resolution=resolution)

# search space
model_space = SimpleAutoencoderSpace()

# model
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet', in_channels=1, out_channels=1, init_features=64, pretrained=False)


# eval module
module = LightningEval(
                model_space, # not sure if space or model goes here
                phantom=phantom, 
                buffer_size=100,
                num_iter=50,
                lr=0.00005, 
                noise_type='gaussian', 
                noise_factor=0.15, 
                resolution=resolution, 
                )
evaluator = FunctionalEvaluator(module)

# search strategy
strategy = DARTS()

# experiment
exp = RetiariiExperiment(model_space, evaluator, [], strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = 'mnist_search'
exp_config.trial_code_directory = 'C:/Users/Public/Public_VS_Code/NAS_test'
exp_config.experiment_working_directory = 'C:/Users/Public/nni-experiments'

exp_config.max_trial_number = 12   # spawn 50 trials at most
exp_config.trial_concurrency = 2  # will run two trials concurrently

exp_config.trial_gpu_number = 1 # will run 1 trial(s) concurrently
exp_config.training_service.use_active_gpu = True

# Execute
exp.run(exp_config, 8081)


exported_arch = experiment.export_top_models(formatter='dict')[0:10]
exported_arch[0]

In [ ]:
experiment = Experiment.connect(8081)
experiment.stop()